# Projeto Parcial Ciência de Dados (N1)

| Nome | RA |
| ---- | -- |
| Enzo Ferroni | 10417100 |
| Luiz Gabriel Profirio Mendes | 10382703 |



## Escolha dos Modelos

---

* Regressão Linear  
A Regressão Linear foi selecionada por ser um modelo clássico e interpretável, capaz de indicar de forma direta a influência de cada característica sonora no sucesso da música. Essa escolha permite observar relações lineares entre variáveis como dançabilidade, energia ou valência e o desempenho da faixa em termos de streams. Além disso, o modelo é eficiente computacionalmente e serve como uma boa linha de base para comparação

* KNN  
Já o K-Nearest Neighbors Regressor foi escolhido por ser um modelo não paramétrico, que se baseia na proximidade entre exemplos para fazer previsões. Em vez de assumir uma relação matemática específica entre os atributos e o número de streams, o KNN prediz o valor com base na média dos exemplos mais similares, capturando possíveis padrões não lineares presentes nos dados. Essa abordagem é especialmente útil quando há agrupamentos de faixas com características sonoras semelhantes que apresentam desempenhos parecidos em popularidade. Ao utilizar modelos com princípios diferentes, é possível avaliar não apenas o desempenho preditivo de cada um, mas também se há padrões sonoros recorrentes que expliquem o sucesso das faixas analisadas.

## Pré-processamento básico

---

In [2]:
# Importa bibliotecas
import pandas as pd

In [ ]:
# Lê CSV
df = pd.read_csv("spotify-2023.csv", encoding="latin1")

# Foi necessário usar o 'encoding' no bloco acima pois estava com o erro
# UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 7250-7251: invalid continuation byte


In [5]:
# Exibe algumas informações básicas sobre o dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

In [6]:
# Verificando os valores presentes em key
print(f"Value counts para a coluna 'key':")
value_counts = df['key'].value_counts()
display(value_counts)
print("\n")

Value counts para a coluna 'key':


,count
key,
C#,120
G,96
G#,91
F,89
B,81
D,81
A,75
F#,73
E,62


In [7]:
# Remove linhas onde key é null
df = df.dropna(subset=['key'])

# Exibe algumas informações básicas sobre o dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 858 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            858 non-null    object
 1   artist(s)_name        858 non-null    object
 2   artist_count          858 non-null    int64 
 3   released_year         858 non-null    int64 
 4   released_month        858 non-null    int64 
 5   released_day          858 non-null    int64 
 6   in_spotify_playlists  858 non-null    int64 
 7   in_spotify_charts     858 non-null    int64 
 8   streams               858 non-null    object
 9   in_apple_playlists    858 non-null    int64 
 10  in_apple_charts       858 non-null    int64 
 11  in_deezer_playlists   858 non-null    object
 12  in_deezer_charts      858 non-null    int64 
 13  in_shazam_charts      817 non-null    object
 14  bpm                   858 non-null    int64 
 15  key                   858 non-null    object


In [8]:
# Remove colunas de serviços que não serão cobertos em nossa análise.
df = df.drop(columns=['in_apple_playlists', 'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts', 'in_shazam_charts'])

# Exibe algumas informações básicas sobre o dataset.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 858 entries, 0 to 952
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            858 non-null    object
 1   artist(s)_name        858 non-null    object
 2   artist_count          858 non-null    int64 
 3   released_year         858 non-null    int64 
 4   released_month        858 non-null    int64 
 5   released_day          858 non-null    int64 
 6   in_spotify_playlists  858 non-null    int64 
 7   in_spotify_charts     858 non-null    int64 
 8   streams               858 non-null    object
 9   bpm                   858 non-null    int64 
 10  key                   858 non-null    object
 11  mode                  858 non-null    object
 12  danceability_%        858 non-null    int64 
 13  valence_%             858 non-null    int64 
 14  energy_%              858 non-null    int64 
 15  acousticness_%        858 non-null    int64 


In [9]:
# Remove linha da música Love Grows (Where My Rosemary Goes), pois há inconsistência nos dados.
index = df[df['track_name'] == "Love Grows (Where My Rosemary Goes)"].index
df = df.drop(index)

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 857 entries, 0 to 952
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            857 non-null    object
 1   artist(s)_name        857 non-null    object
 2   artist_count          857 non-null    int64 
 3   released_year         857 non-null    int64 
 4   released_month        857 non-null    int64 
 5   released_day          857 non-null    int64 
 6   in_spotify_playlists  857 non-null    int64 
 7   in_spotify_charts     857 non-null    int64 
 8   streams               857 non-null    object
 9   bpm                   857 non-null    int64 
 10  key                   857 non-null    object
 11  mode                  857 non-null    object
 12  danceability_%        857 non-null    int64 
 13  valence_%             857 non-null    int64 
 14  energy_%              857 non-null    int64 
 15  acousticness_%        857 non-null    int64 


In [10]:
# Converte streams para numérico
df['streams'] = pd.to_numeric(df['streams'])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 857 entries, 0 to 952
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            857 non-null    object
 1   artist(s)_name        857 non-null    object
 2   artist_count          857 non-null    int64 
 3   released_year         857 non-null    int64 
 4   released_month        857 non-null    int64 
 5   released_day          857 non-null    int64 
 6   in_spotify_playlists  857 non-null    int64 
 7   in_spotify_charts     857 non-null    int64 
 8   streams               857 non-null    int64 
 9   bpm                   857 non-null    int64 
 10  key                   857 non-null    object
 11  mode                  857 non-null    object
 12  danceability_%        857 non-null    int64 
 13  valence_%             857 non-null    int64 
 14  energy_%              857 non-null    int64 
 15  acousticness_%        857 non-null    int64 


## Implementação

---

### Regressão Linear

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Seleção de variáveis
features = ['bpm', 'danceability_%', 'valence_%', 'energy_%', 'acousticness_%',
            'instrumentalness_%', 'liveness_%', 'speechiness_%']
X = df[features]
y = df['streams']

# Separação entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo
reg = LinearRegression()
reg.fit(X_train, y_train)

# Previsão
y_pred = reg.predict(X_test)

### KNN

In [12]:
from sklearn.neighbors import KNeighborsRegressor

# Modelo
knn = KNeighborsRegressor(n_neighbors=5, weights='uniform')
knn.fit(X_train, y_train)

# Previsão
y_pred_knn = knn.predict(X_test)

## Métricas

---

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Avaliação - Regressão Linear
mae_lr = mean_absolute_error(y_test, y_pred)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred))
r2_lr = r2_score(y_test, y_pred)

print("Regressão Linear:")
print(f"MAE: {mae_lr:.2f}")
print(f"RMSE: {rmse_lr:.2f}")
print(f"R²: {r2_lr:.2f}")

# Avaliação - KNN Regressor
mae_knn = mean_absolute_error(y_test, y_pred_knn)
rmse_knn = np.sqrt(mean_squared_error(y_test, y_pred_knn))
r2_knn = r2_score(y_test, y_pred_knn)

print("\nKNN Regressor:")
print(f"MAE: {mae_knn:.2f}")
print(f"RMSE: {rmse_knn:.2f}")
print(f"R²: {r2_knn:.2f}")


Regressão Linear:
MAE: 467433722.69
RMSE: 676784872.64
R²: -0.00

KNN Regressor:
MAE: 506617278.06
RMSE: 736012757.92
R²: -0.19


## Comparação

---

## Conclusão preliminar

---